# Correlate variables to remove redundant metrics

In [1]:
suppressPackageStartupMessages({
    library(here)
    library(dplyr)
    library(SummarizedExperiment)
})

## Functions

In [2]:
load_phenotypes <- function(region){
    pheno_file <- here("input/phenotypes/_m/phenotypes.csv")
    pheno = data.table::fread(pheno_file) |> filter(Region == region) |> 
        mutate_if(is.list, ~sapply(., sum)) |>
        mutate_if(is.numeric, scales::rescale)
    return(pheno)
}

check_dup <- function(df){
    sample    <- df |> select_if(is.numeric)
    sample    <- Filter(function(x) sd(x) != 0, sample)
    variables <- names(sample)
    return(cytominer::correlation_threshold(variables, sample, cutoff=0.95))
}

check_corr <- function(df){
    sample <- df |> select_if(is.numeric)
    sample <- Filter(function(x) sd(x) != 0, sample)
    dt     <- sample |> corrr::correlate() |>
        corrr::stretch() |> tidyr::drop_na() |>
        filter(abs(r) > 0.95) |>
        distinct(r, .keep_all=TRUE)
    varX <- distinct(dt, x)$x
    varX <- varX[-which(varX %in% intersect(varX, distinct(dt, y)$y))]
    vars <- unique(c(distinct(dt, x)$x, distinct(dt, y)$y))
    return(setdiff(vars, varX))
}

remove_variables <- function(pheno_df){
    if(length(check_corr(pheno_df)) != 0){
        pheno_df <- pheno_df |> select(-check_corr(pheno_df))
    }
    return(pheno_df)
}

## Main

### Load phenotypes

In [3]:
caudate <- load_phenotypes("Caudate")
dlpfc   <- load_phenotypes("DLPFC")
hippo   <- load_phenotypes("HIPPO")

In [4]:
caudate |> dim()
dlpfc |> dim()
hippo |> dim()

[1] 418  57

[1] 388  57

[1] 408  57

### Drop correlated

In [5]:
check_corr(caudate)
check_corr(dlpfc)
check_corr(hippo)

Correlation computed with
• Method: 'pearson'
• Missing treated using: 'pairwise.complete.obs'


[1] "Exonic_Rate"         "gene_Assigned"       "Intronic_Rate"      
[4] "End_2_Sense_Rate"    "Mapped_Unique_Reads" "NonGlobin_Reads"

Correlation computed with
• Method: 'pearson'
• Missing treated using: 'pairwise.complete.obs'


[1] "Exonic_Rate"         "Low_Quality_Reads"   "gene_Assigned"      
 [4] "Ambiguous_Reads"     "Intronic_Reads"      "NonGlobin_Reads"    
 [7] "Intronic_Rate"       "End_2_Sense_Rate"    "numReads"           
[10] "Mapped_Unique_Reads" "MedianAvgTxCov"

Correlation computed with
• Method: 'pearson'
• Missing treated using: 'pairwise.complete.obs'


[1] "Exonic_Rate"         "gene_Assigned"       "Intronic_Rate"      
[4] "mitoRate"            "End_2_Sense_Rate"    "Low_Quality_Reads"  
[7] "Mapped_Unique_Reads" "NonGlobin_Reads"     "Intronic_Reads"

In [6]:
caudate <- remove_variables(caudate)
dlpfc   <- remove_variables(dlpfc)
hippo   <- remove_variables(hippo)

Correlation computed with
• Method: 'pearson'
• Missing treated using: 'pairwise.complete.obs'
Correlation computed with
• Method: 'pearson'
• Missing treated using: 'pairwise.complete.obs'
Correlation computed with
• Method: 'pearson'
• Missing treated using: 'pairwise.complete.obs'
Correlation computed with
• Method: 'pearson'
• Missing treated using: 'pairwise.complete.obs'
Correlation computed with
• Method: 'pearson'
• Missing treated using: 'pairwise.complete.obs'
Correlation computed with
• Method: 'pearson'
• Missing treated using: 'pairwise.complete.obs'


In [7]:
caudate |> dim()
dlpfc |> dim()
hippo |> dim()

[1] 418  51

[1] 388  46

[1] 408  48

### Commone variables

In [8]:
vars <- intersect(colnames(caudate),intersect(colnames(dlpfc), colnames(hippo)))
vars

[1] "SAMPLE_ID"                "RNum"                    
 [3] "Region"                   "Dataset"                 
 [5] "Protocol"                 "RIN"                     
 [7] "BrNum"                    "Dx"                      
 [9] "Race"                     "Sex"                     
[11] "Age"                      "PMI"                     
[13] "MoD"                      "Mapping_Rate"            
[15] "Base_Mismatch"            "ExprProfEff"             
[17] "Intergenic_Rate"          "totalAssignedGene"       
[19] "Ambiguous_Alignment_Rate" "rRNA_rate"               
[21] "End_1_Sense_Rate"         "Chimeric_Alignment_Rate" 
[23] "Low_Mapping_Quality"      "Genes_Detected"          
[25] "Mean3Bias"                "totalMapped"             
[27] "Intergenic_Reads"         "Read_Length"             
[29] "Mito_mapped"              "globinRate"              
[31] "IID"                      "SOL"                     
[33] "snpPC1"                   "snpPC2"                  
[35] "snpPC3"                   "snpPC4"                  
[37] "snpPC5"                   "snpPC6"                  
[39] "snpPC7"                   "snpPC8"                  
[41] "snpPC9"                   "snpPC10"                 
[43] "New_Dx"                   "antipsychotics"          
[45] "lifetime_antipsych"

In [9]:
length(vars)

[1] 45

In [10]:
data.frame("Variables"=vars) |>
    data.table::fwrite("shared_variables.tsv", sep='\t')

## Reproducibility

In [11]:
Sys.time()
proc.time()
options(width = 120)
sessioninfo::session_info()$platform
sessioninfo::session_info()$packages

[1] "2023-02-13 18:45:00 EST"

   user  system elapsed 
  5.183   0.195   7.075 

setting  value
 version  R version 4.2.2 (2022-10-31)
 os       Arch Linux
 system   x86_64, linux-gnu
 ui       X11
 language (EN)
 collate  en_US.UTF-8
 ctype    en_US.UTF-8
 tz       America/New_York
 date     2023-02-13
 pandoc   3.0.1 @ /usr/bin/pandoc

,package,ondiskversion,loadedversion,path,loadedpath,attached,is_base,date,source,md5ok,library
,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<chr>,<chr>,<lgl>,<fct>
base64enc,base64enc,0.1.3,0.1-3,/home/kynon/R/x86_64-pc-linux-gnu-library/4.2/base64enc,/home/kynon/R/x86_64-pc-linux-gnu-library/4.2/base64enc,FALSE,FALSE,2015-07-28,CRAN (R 4.2.1),NA,/home/kynon/R/x86_64-pc-linux-gnu-library/4.2
Biobase,Biobase,2.58.0,2.58.0,/home/kynon/R/x86_64-pc-linux-gnu-library/4.2/Biobase,/home/kynon/R/x86_64-pc-linux-gnu-library/4.2/Biobase,TRUE,FALSE,2022-11-01,Bioconductor,NA,/home/kynon/R/x86_64-pc-linux-gnu-library/4.2
BiocGenerics,BiocGenerics,0.44.0,0.44.0,/home/kynon/R/x86_64-pc-linux-gnu-library/4.2/BiocGenerics,/home/kynon/R/x86_64-pc-linux-gnu-library/4.2/BiocGenerics,TRUE,FALSE,2022-11-01,Bioconductor,NA,/home/kynon/R/x86_64-pc-linux-gnu-library/4.2
bitops,bitops,1.0.7,1.0-7,/home/kynon/R/x86_64-pc-linux-gnu-library/4.2/bitops,/home/kynon/R/x86_64-pc-linux-gnu-library/4.2/bitops,FALSE,FALSE,2021-04-24,CRAN (R 4.2.2),NA,/home/kynon/R/x86_64-pc-linux-gnu-library/4.2
cli,cli,3.6.0,3.6.0,/home/kynon/R/x86_64-pc-linux-gnu-library/4.2/cli,/home/kynon/R/x86_64-pc-linux-gnu-library/4.2/cli,FALSE,FALSE,2023-01-09,CRAN (R 4.2.2),NA,/home/kynon/R/x86_64-pc-linux-gnu-library/4.2
colorspace,colorspace,2.1.0,2.1-0,/home/kynon/R/x86_64-pc-linux-gnu-library/4.2/colorspace,/home/kynon/R/x86_64-pc-linux-gnu-library/4.2/colorspace,FALSE,FALSE,2023-01-23,CRAN (R 4.2.2),NA,/home/kynon/R/x86_64-pc-linux-gnu-library/4.2
corrr,corrr,0.4.4,0.4.4,/home/kynon/R/x86_64-pc-linux-gnu-library/4.2/corrr,/home/kynon/R/x86_64-pc-linux-gnu-library/4.2/corrr,FALSE,FALSE,2022-08-16,CRAN (R 4.2.2),NA,/home/kynon/R/x86_64-pc-linux-gnu-library/4.2
crayon,crayon,1.5.2,1.5.2,/home/kynon/R/x86_64-pc-linux-gnu-library/4.2/crayon,/home/kynon/R/x86_64-pc-linux-gnu-library/4.2/crayon,FALSE,FALSE,2022-09-29,CRAN (R 4.2.2),NA,/home/kynon/R/x86_64-pc-linux-gnu-library/4.2
data.table,data.table,1.14.6,1.14.6,/home/kynon/R/x86_64-pc-linux-gnu-library/4.2/data.table,/home/kynon/R/x86_64-pc-linux-gnu-library/4.2/data.table,FALSE,FALSE,2022-11-16,CRAN (R 4.2.2),NA,/home/kynon/R/x86_64-pc-linux-gnu-library/4.2
